# Attempting to Size the Psychiatry Specialty
For context: 

The AAMC https://www.aamc.org/data/workforce/reports/492556/1-1-chart.html lists  892,856 in total active specialty physicans and 38,205 (4.3%) psychiatrists for 2017.

Statista here https://www.statista.com/statistics/209424/us-number-of-active-physicians-by-specialty-area/ lists 525,439 total active "specialty" physicians in March of 2019 of which 54,935 (10.5%) are psychiatrists. 

Keeping in mind there's a shortage of psychiatrists in general: https://news.aamc.org/patient-care/article/addressing-escalating-psychiatrist-shortage/

Here, I'm going to try to scrape individuals from the NPI registry to see how many individuals are listed under the taxonomy of 'psychiatry'



In [2]:
import pandas as pd
import requests
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import re
import time
from bs4 import BeautifulSoup

In [65]:
#have to skip every 200 
skip=range(0,20000,200)

In [77]:
# #used this to try and see if there were more than 1200 
# skip=[1200,2400]

In [74]:
data=[]

In [75]:
#individual test
# link=f'https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=NPI-2&taxonomy_description=Psychiatry&first_name=&last_name=&organization_name=&address_purpose=&city=&state=&postal_code=&country_code=&limit=200&skip=300&version=2.0'
# response_npi = requests.get(link)
# data_npi = response_npi.json()

In [78]:
for num in skip:
    link=f'https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=NPI-2&taxonomy_description=Psychiatry&first_name=&last_name=&organization_name=&address_purpose=&city=&state=&postal_code=&country_code=&limit=200&skip={num}&version=2.1'
    response_npi = requests.get(link)
    data_npi = response_npi.json()
    results=data_npi['results']
    print('---------------------------------')
    print(results[0]['basic']['authorized_official_first_name'])
    print(len(results))
#     time.sleep(1)
    for result in results[:len(results)-1]:
        person=result['basic']
        print(result['basic']['authorized_official_first_name'])
        for address in result['addresses']:
            column_name='zip_'+ str(result['addresses'].index(address)+1)
            person[column_name]=address['postal_code']
#             data.append(person)

---------------------------------
CHUCK
200
CHUCK
DAVID
MARK
DAVID
ALBERTA
HATTI
MICKEY
NANCY
DARLENE
NEIL
MANUEL
MANOJ
RALPH
PAUL
SWAMPNA
GAIL
ANDREW
JUDE
JACQUELINE
BRONSON
LISA
MONICA
NORMAN
TROY
TALIN
MELISSA
STEVEN
BRYON
RANDOLPH
DEBORAH
JENNIFER
PENELOPE
BENNETT
FREDERICK
FREDERICK
FREDERICK
FREDERICK
MICHAEL
KEITH
MICHAEL
BENNETT
RICHARD
DANIEL
LEISA
STEVE
HITEN
ANTHONY
LUCKOSE
THOMAS
LAWRENCE
REBECCA
ANGELA
KAREN
FRANCES
EDWIN
LINDA
WILLIAM
HOWARD
FRANCISCO
PATRICK
FREDERICK
CRYSTAL
RICK
MAXWELL
D
D
D
ROBERT
KENNETH
DAVID
REBECCA
EDWARD
AMY
CHARLES
JOSEPH
ERIC
ROGER
LEFAWN
CHERYL
GREG
DAVIS
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
MELISSA
DAVE
WALTER
MICHAEL
CARL
LOUISE
DENNIS
JASON
OMAR
JORRI
MICHAEL
JEFFREY
WILLIAM
JILL
MELISSA
ANTHONY
JEROME
CRAIG
SUSAN
JASMINE
DARREL
CANDACE
ABDUL
ALEXANDRA
STEPHEN
ALBERTO
LARRY
ROYAL
FEBE
GEOFFREY
RUSSELL
RUTH
MONIQUE
SYAM
JOSH
CHRIS
IAN
DEBORAH
ROBIN
FREDERICK
NITIN
ANN MARE
TIMOTHY
R

In [70]:
df=pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39800 entries, 0 to 39799
Data columns (total 20 columns):
authorized_official_credential             19834 non-null object
authorized_official_first_name             39800 non-null object
authorized_official_last_name              39800 non-null object
authorized_official_middle_name            19854 non-null object
authorized_official_name_prefix            18482 non-null object
authorized_official_name_suffix            1016 non-null object
authorized_official_telephone_number       39800 non-null object
authorized_official_title_or_position      39800 non-null object
deactivation_date                          2106 non-null object
enumeration_date                           39800 non-null object
last_updated                               39800 non-null object
name                                       39800 non-null object
organization_name                          39800 non-null object
organizational_subpart                     39800

In [71]:
df.duplicated().value_counts()

True     38606
False     1194
dtype: int64

In [79]:
df=df.drop_duplicates(keep='first')

In [81]:
df.shape

(1194, 20)

So, turns out there are only 1,194 Psychiatrists and Neurologists listed which is an estimated: 2-3% of the workforce. 

In [82]:
df.to_csv('npi_psychiatrists.csv',index=False)

In [4]:
names=pd.read_csv('/Users/lilyliu-krason/Documents/lede/foundations/data_studio_projects/psychiatrists_diversity/names/Names_2010Census.csv')

In [7]:
df=pd.read_csv('npi_psychiatrists.csv')

In [14]:
npi_names=pd.DataFrame(df.authorized_official_last_name.value_counts().reset_index().rename(columns={'index':'name','authorized_official_last_name':'count'}))

In [16]:
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162254 entries, 0 to 162253
Data columns (total 11 columns):
name            162253 non-null object
rank            162254 non-null int64
count           162254 non-null int64
prop100k        162254 non-null float64
cum_prop100k    162254 non-null float64
pctwhite        162254 non-null object
pctblack        162254 non-null object
pctapi          162254 non-null object
pctaian         162254 non-null object
pct2prace       162254 non-null object
pcthispanic     162254 non-null object
dtypes: float64(2), int64(2), object(7)
memory usage: 13.6+ MB


In [17]:
merge=npi_names.merge(names,left_on='name',right_on='name',how='left')

In [18]:
merge

,name,count_x,rank,count_y,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,KING,19,34.0,465422.0,157.78,10347.92,70.16,22.76,1.23,0.98,2.31,2.55
1,FALVO,17,26313.0,928.0,0.31,77763.40,95.69,0.54,0.75,0,1.72,1.29
2,BROWN,15,4.0,1437026.0,487.16,2521.56,57.95,35.6,0.51,0.87,2.55,2.52
3,MCGINTY,10,5697.0,6084.0,2.06,61994.83,88.25,6.33,0.61,0.84,1.61,2.37
4,AILES,9,38902.0,567.0,0.19,80834.59,88.89,6.35,(S),(S),1.23,2.47
5,SMITH,8,1.0,2442977.0,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4
6,CERISE,7,52619.0,394.0,0.13,83032.39,91.88,1.78,(S),(S),2.03,3.55
7,FRIEDMAN,5,901.0,38374.0,13.01,39740.92,96.05,0.41,0.6,0.13,0.94,1.88
8,FILTON,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ADAMS,5,42.0,427865.0,145.05,11534.72,74.02,19.9,0.56,0.79,2.16,2.57
